In [1]:
from aqua.lra_generator import LRA_Generator
from aqua.reader import Reader

In [3]:
model = 'IFS'
exp = 'tco3999-ng5'
source = 'ICMGG_atm2d'

varlist = ['msl','tsr']

outdir = '/scratch/b/b382289/lra_test'

In [19]:
read = Reader(model=model, exp=exp, source=source, configdir="../../config", var=varlist)

In [20]:
read.retrieve(fix=False)

<xarray.Dataset>
Dimensions:  (time: 5401, value: 64144000)
Coordinates:
    lat      (value) float64 dask.array<chunksize=(64144000,), meta=np.ndarray>
    lon      (value) float64 dask.array<chunksize=(64144000,), meta=np.ndarray>
  * time     (time) datetime64[ns] 2020-01-20 2020-01-20T01:00:00 ... 2020-09-01
Dimensions without coordinates: value
Data variables:
    msl      (time, value) float64 dask.array<chunksize=(1, 64144000), meta=np.ndarray>
    tsr      (time, value) float64 dask.array<chunksize=(1, 64144000), meta=np.ndarray>
Attributes:
    edition:            1
    centre:             ecmf
    centreDescription:  European Centre for Medium-Range Weather Forecasts
    subCentre:          0
    history:            🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan.IF...

In [4]:
test = LRA_Generator(model=model,exp=exp,source=source,varlist=varlist,outdir=outdir,
                     resolution='r100',dry=True,verbose=True,fix=False)

IMPORTANT: no file will be created, this is a dry run
Variables to be processed: ['msl', 'tsr']
Frequency not specified, streaming mode
Fixing data: False
Folder /scratch/b/b382289/lra_test/tco3999-ng5/r100 already exists


In [5]:
test.retrieve_data()

Accessing catalog for IFS-tco3999-ng5-ICMGG_atm2d...
I am going to produce LRA at r100 resolution and None frequency...
Retrieving data...
<xarray.Dataset>
Dimensions:  (time: 5401, value: 64144000)
Coordinates:
    lat      (value) float64 dask.array<chunksize=(64144000,), meta=np.ndarray>
    lon      (value) float64 dask.array<chunksize=(64144000,), meta=np.ndarray>
  * time     (time) datetime64[ns] 2020-01-20 2020-01-20T01:00:00 ... 2020-09-01
Dimensions without coordinates: value
Data variables:
    msl      (time, value) float64 dask.array<chunksize=(1, 64144000), meta=np.ndarray>
    tsr      (time, value) float64 dask.array<chunksize=(1, 64144000), meta=np.ndarray>
Attributes:
    edition:            1
    centre:             ecmf
    centreDescription:  European Centre for Medium-Range Weather Forecasts
    subCentre:          0
    history:            🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan.IF...


In [6]:
test.generate_lra()

Generating LRA data...
Processing variable msl...
Processing year 2020...
Processing month 1...
Processing month 2...
Processing month 3...
Processing month 4...
Processing month 5...
Processing month 6...
Processing month 7...
Processing month 8...
Processing month 9...
Process took 0.6137 seconds
Processing variable tsr...
Processing year 2020...
Processing month 1...
Processing month 2...
Processing month 3...
Processing month 4...
Processing month 5...
Processing month 6...
Processing month 7...
Processing month 8...
Processing month 9...
Process took 0.1000 seconds
Finished generating LRA data for IFS-tco3999-ng5-ICMGG_atm2d at r100 resolution and None frequency.
